# TP : Spark MLIb

Prise en main de Spark MLib via un use case simple de détection de churn.

In [1]:
import pandas as pd
import numpy as np

from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression

# Generate fake customer churn data
data = {
    'customer_id': np.arange(1, 10001),
    'age': np.random.randint(18, 70, 10000),
    'contract_type': np.random.choice(['Month-to-month', 'One year', 'Two year'], 10000),
    'monthly_charges': np.round(np.random.uniform(20, 100, 10000), 2),
    'total_charges': np.round(np.random.uniform(500, 5000, 10000), 2),
    'internet_service': np.random.choice(['DSL', 'Fiber optic', 'None'], 10000),
    'churn': np.random.choice([0, 1], 10000)  # Binary target variable
}
df = pd.DataFrame(data)

In [2]:
df.head()

,customer_id,age,contract_type,monthly_charges,total_charges,internet_service,churn
0,1,66,One year,30.22,1629.86,DSL,1
1,2,32,Two year,25.59,1132.49,DSL,1
2,3,48,One year,45.52,4360.97,None,0
3,4,51,Month-to-month,89.88,2027.26,None,0
4,5,38,Month-to-month,84.78,4542.47,None,1


In [3]:
df.describe()

,customer_id,age,monthly_charges,total_charges,churn
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000
mean,5000.50000,43.645400,59.829156,2740.082005,0.501700
std,2886.89568,15.045315,23.085017,1298.224201,0.500022
min,1.00000,18.000000,20.010000,500.260000,0.000000
25%,2500.75000,31.000000,39.857500,1625.982500,0.000000
50%,5000.50000,44.000000,59.490000,2748.355000,1.000000
75%,7500.25000,57.000000,79.902500,3840.297500,1.000000
max,10000.00000,69.000000,99.950000,4999.990000,1.000000


TODO: convert to Spark Dataframe

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Churn prediction").getOrCreate()

In [5]:
#customer_df =

In [6]:
# String index categorical columns
contract_indexer = StringIndexer(inputCol="contract_type", outputCol="contract_index")
internet_indexer = StringIndexer(inputCol="internet_service", outputCol="internet_index")

# Assemble features into a single vector
assembler = VectorAssembler(inputCols=["age", "contract_index", "monthly_charges", "total_charges", "internet_index"], outputCol="features")

pipeline = Pipeline(stages=[contract_indexer, internet_indexer, assembler])

TODO: apply pipeline to data (fit, transform)

In [7]:
#customer_df_transformed =

In [8]:
# Split data
train, test = customer_df_transformed.randomSplit([0.8, 0.2], seed=1234)

# Define and train the model
lr = LogisticRegression(featuresCol="features", labelCol="churn")
lr_model = lr.fit(train)

# Evaluate the model
predictions = lr_model.transform(test)
predictions.select("customer_id", "features", "churn", "prediction").show(5)

+-----------+--------------------+-----+----------+
|customer_id|            features|churn|prediction|
+-----------+--------------------+-----+----------+
|          2|[32.0,1.0,25.59,1...|    1|       1.0|
|          5|[38.0,0.0,84.78,4...|    1|       1.0|
|         22|[51.0,1.0,38.1,42...|    1|       1.0|
|         25|[29.0,2.0,40.4,30...|    0|       0.0|
|         26|[48.0,2.0,96.12,3...|    1|       1.0|
+-----------+--------------------+-----+----------+
only showing top 5 rows



TODO: experiment with other models, such as RandomForestClassifier, and compare metrics like accuracy or precision.

https://spark.apache.org/docs/latest/ml-guide